In [ ]:
import sys
import numpy as np
sys.path.append('..')
from Comms_System import Comms_System
from train_decision_making import train_DM_model
#from DE_minibatch import DE
from DE import DE
from objective_functions import MSE, crossEntropy, crossEntropyPytorch
from Network import NeuralNetwork
import matplotlib.pyplot as plt
import scipy.stats
from objective_functions import stablesoftmax as softmax

# Train Model

In [ ]:
symbol_set = [3, 1, -1, -3] # all symbols that we use
num_symbols = 2000
SNRdb = 10
symbol_seq = np.random.choice(symbol_set, num_symbols, replace=True)
m = 8
CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m)
sigma = CS.SNRdb_to_sigma(SNRdb, use_gain=True)
print(sigma)
# calibrate
gain_factor = np.max(np.convolve(CS.h, CS.h))

# upsample symbol sequence and filter it on transmission side
upsampled = CS.upsample()
Tx = np.convolve(upsampled, CS.h)

# Transmit the filtered signal (i.e. add noise)
Tx = Tx + np.random.normal(0.0, sigma, Tx.shape)  # add gaussian noise

# Filter on receiver side
Rx = np.convolve(Tx, CS.h)#/np.sqrt(gain_factor)

# Downsample the signal on the receiver side
downsampled = CS.downsample(Rx)/gain_factor

X = np.array(downsampled, ndmin=2).T    
y = symbol_seq
classes = np.array(symbol_set)
num_classes = len(classes)
sizes = [1, 8, 4]
sizes[-1] = num_classes

# Makes a dictionary that maps a number to each symbol e.g. 3: 0
class_idx = {v: i for i, v in enumerate(classes)}

# Maps class indexes to each value in targets
y = np.array([class_idx[v] for v in y])

# Converts to one-hot-encoded
y = np.eye(num_classes)[y]


# split train-test
splitlen = int(X.shape[0] * 0.5)
Xtrain, ytrain = X[:splitlen], y[:splitlen]
Xtest, ytest = X[splitlen:], y[splitlen:]

In [ ]:
# Inspect symbol distribution in training (downsampled) data
plt.figure(figsize=(13,8))
for symbol in symbol_set:

    idx_ = symbol_seq[:splitlen] == symbol
    plt.hist(Xtrain[idx_], label=str(symbol))
    
plt.xlabel('Downsampled Values', fontsize=22)
plt.ylabel('Frequency', fontsize=22)
plt.legend(fontsize=14)
plt.show()
#plt.savefig('SymbolDistribution', bbox_inches='tight', transparent=True)

In [ ]:
optimizer = 'DE'
early_stop = False

if optimizer == 'DE':
    D = DE(objective_function=crossEntropy, sizes=sizes, pop_size=50, F=0.55, cr=0.85,
           X=Xtrain, y=ytrain, Xtest=Xtest, ytest=ytest, type='classification', afunc='relu')
    
    if early_stop:
        best_agent = D.early_stop_training(patience=100)
    else:
        best_agent = D.evolution(num_epochs=5001, verbose=True, print_epoch=500)
        D.evaluate()
    
elif optimizer == 'BP':
    NN = NeuralNetwork(sizes=sizes, type='classification', afunc='relu')
    NN.train_loop(Xtrain, ytrain, Xtest, ytest, epochs=10000, batch_size=Xtrain.shape[0], 
                  cost=True, acc=True, plot=True)
    best_agent = NN

# Prediction Accuracy

In [ ]:
best_agent = D.evolution(num_epochs=1001, verbose=True, print_epoch=500)
D.evaluate()

In [ ]:
# Get predictions
predictions = best_agent.feedforward(Xtest).argmax(axis=1)
true = ytest.argmax(axis=1)

# Convert back to symbols
predicted_symbols = classes[predictions]
true_values = classes[true]
# print(predicted_symbols, true_values)
correct_preds = np.equal(true_values, predicted_symbols)
print("Accuracy: ", (sum(correct_preds) / len(true_values)) * 100, "%")

# Error Rate

In [ ]:
symbol_set = [3, 1, -1, -3] # all symbols that we use
num_symbols = 100000
symbol_seq = np.random.choice(symbol_set, num_symbols, replace=True)
m = 8
SNRdb = 10

CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m)
euclid_decisions, NN_decisions = CS.transmit_all_same(SNRdb=SNRdb, NN_model=NN, block=False)
print('Euclidean Distance Error Rate: {}%'.format((CS.evaluate(euclid_decisions)[1]*100).round(2)))
print('Neural Net Error Rate: {}%'.format((CS.evaluate(NN_decisions)[1]*100).round(2)))

In [ ]:
symbol_set = [3, 1, -1, -3]  # all symbols that we use
symbol_seq = np.random.choice(symbol_set, 10000, replace=True)
CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=8, beta=0.35)
SNRdbs = np.linspace(0, 19, 50)
euclid_error_rates = np.zeros(len(SNRdbs))
NN_error_rates = np.zeros(len(SNRdbs))
avg_symbol_energy = np.mean(symbol_seq ** 2)
print('Avg symbol energy', avg_symbol_energy)
gain_factor = np.max(np.convolve(CS.h, CS.h))
print('gain', gain_factor)

for i, SNRdb in enumerate(SNRdbs):
    euclid_decisions, NN_decisions = CS.transmit_all_same(SNRdb=SNRdb, NN_model=NN, block=False)
    euclid_error_rates[i] = CS.evaluate(euclid_decisions)[1]
    NN_error_rates[i] = CS.evaluate(NN_decisions)[1]

sigmas = np.array([CS.SNRdb_to_sigma(SNRdb, avg_symbol_energy, use_gain=True) for SNRdb in SNRdbs])
error_theory = 1.5 * (1 - scipy.stats.norm.cdf(np.sqrt(gain_factor / sigmas ** 2)))

In [ ]:
plt.figure(figsize=(13,8))
plt.xlabel('SNR (dB)', fontsize=20)
plt.ylabel('$P_e$', fontsize=24)
num = 0
plt.semilogy(SNRdbs[num:], euclid_error_rates[num:], alpha=1, linewidth=3, label='Euclidean')
plt.semilogy(SNRdbs[num:], NN_error_rates[num:], alpha=0.75, linewidth=3, label='NN Dec Maker')
plt.semilogy(SNRdbs[num:], error_theory[num:], alpha=0.75, linewidth=3, label='Theory')
plt.legend(fontsize=16)
plt.show()
#plt.savefig('NN_Dec_Performance4')

In [ ]:
test_seq = np.linspace(-4, 4, 40)
test_seq = np.array(test_seq, ndmin=2).T
yhat_onehot = D.best_agent.feedforward(test_seq)


plt.figure(figsize=(13,8))
#plt.title('Decision Boundaries', fontsize=24)
for i in range(yhat_onehot.shape[1]):
    plt.plot(test_seq, yhat_onehot[:,i], '-x')
#plt.axvline(x=-2, color='black')
#plt.axvline(x=0, color='black')
#plt.axvline(x=2, color='black')
plt.xlabel('Downsampled', fontsize=20)
plt.ylabel('Probability of choosing', fontsize=20)
plt.legend(symbol_set + ['Euclidean Boundaries'], fontsize=14)
plt.savefig('Boundaries2', bbox_inches='tight', transparent=True)

----

# How to retrain automatically

In [ ]:
train = False
if train:
    D = train_DM_model(sizes=[1,8,4], num_symbols=1000, epochs=2001)

In [ ]:
# Evaluate
symbol_set = [3, 1, -1, -3] # all symbols that we use
num_symbols = 10000
symbol_seq = np.random.choice(symbol_set, num_symbols, replace=True)
m = 8
CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m)
noise_level = 1

decisions, _ , _, downsampled = CS.test_CS(noise_level=noise_level, v=False)
X = np.array(downsampled, ndmin=2).T
classes = np.array(symbol_set)
new_NN_decisions = classes[D.best_agent.feedforward(X).argmax(axis=1)]

print('Euclidean Distance Error Rate: {}%'.format((CS.evaluate(decisions)[1]*100).round(2)))
print('Neural Net Error Rate: {}%'.format((CS.evaluate(new_NN_decisions)[1]*100).round(2)))

# Does model perform better? Then save the weights

In [ ]:
save_weights = False

if save_weights:
    D.save_params('decision_making_weights_CE_new3', 'decision_making_biases_CE_new3')